In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull

import itertools
from sklearn.metrics import silhouette_score
from scipy.stats import f_oneway, ttest_rel, ttest_ind
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.anova import AnovaRM

import sys, os
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

# Custom modules
from wholebrain_tools import aba, dataIO
import wholebrain_tools.graphics as gt

# You can provide a "structures.json" file. If you dont, it will download one
paths = dataIO.pathParser()
structuresFile = paths.structures
A = aba.Atlas(nodes=structuresFile)
DFM = aba.AnatomyDataFrameManager(A)

# Load data for WFA

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
channelName = 'wfa'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# Dataframe at mid-ontology
midDf_wfa = DFM.regionsDf_to_mid(df, verbose=False, normalize=True)
# Select only cortical areas
cortex_df = midDf_wfa.loc[315]

# Calculate mean and sem
mean = cortex_df.groupby('params', axis=1).mean()
sem = cortex_df.groupby('params', axis=1).sem().add_suffix('_sem')

# Get the annotation of cortical areas in subnetworks
subnet = dataIO.corticalAreasGroups(toDataFrame=True)
subnet.index = A.acronyms_to_ids(subnet.index)

data = pd.concat([mean, sem, subnet], axis=1)

# Drop areas with unknown function ("other")
data = data.drop(data[data['function']=='other'].index)

# Rename values in "function" columns for aesthetic purposes
switchDict = {
    'mAssociation':'Medial Association',
    'mPrefrontal': 'Medial Prefrontal',
    'lateral':'Lateral',
    'motorSS':'Motor-Somatosensory',
    'audioVisual':'Audio-Visual'
}
data['function_nice'] = [switchDict[x] for x in data['function']]

data.head()

# Plot the cortical subnetworks

In [ ]:
sns.set_style('white')

f, ax = plt.subplots(figsize=(7,7))

# Main scatterplot
g = sns.scatterplot(
    data=data,
    x='diffuseFluo',
    y='energy',
    s=140,
    hue='function_nice'
)

# Add the convex hull background
bgPalette = sns.color_palette("pastel")
for i, areacategory in enumerate(data['function'].unique()):
    set = data.loc[data['function']==areacategory, ['diffuseFluo', 'energy']]
    points = set.values
    hull = ConvexHull(points)
    p = Polygon(points[hull.vertices,:], alpha=0.3, zorder=-20, facecolor=bgPalette[i])
    ax.add_patch(p)

# Plot the errorbars
plt.errorbar(
    x=data['diffuseFluo'],
    y=data['energy'],
    xerr=data['diffuseFluo_sem'],
    yerr=data['energy_sem'],
    fmt='None',
    ecolor='gray',
    elinewidth=.5,
    zorder=-13
)

plt.legend(
    title='Cortical Subnetwork',
    fontsize=14,
    loc='best',
    title_fontsize=16,
    frameon=False
)

# Customize Axis
g.set_xlabel("WFA Diffuse Fluorescence (A.U.)", fontsize=22)
g.set_ylabel("PNN Energy (A.U.)", fontsize=22)
ax.tick_params(labelsize=16)

sns.despine()

# plt.savefig('CorticalSubnetworks.svg',bbox_inches="tight")

# Quantify cluster quality with silhouette score

## Load data

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
# --------------------------------------------------------------------

# WFA
df_wfa = dataIO.allMiceRegions(searchPath=searchPath, channelName='wfa', normCellIntens=True)
df_wfa = DFM.multiIndexDf_from_fineDf(df_wfa, verbose=False)

# Dataframe at mid-ontology
midDf_wfa = DFM.regionsDf_to_mid(df_wfa, verbose=False, normalize=True)

midDf_wfa

## Perform some preprocessing

In [ ]:
# Select cortical layers
allAnimals = midDf_wfa.reorder_levels([1, 0], axis=1).loc[315,['diffuseFluo','energy']]

# Get the annotation of cortical areas in subnetworks
subnet = dataIO.corticalAreasGroups(toDataFrame=True)
subnet.index = A.acronyms_to_ids(subnet.index)

newIndex = [('X', x) for x in subnet.columns.tolist()]
subnet.columns = newIndex
temp = pd.concat([allAnimals, subnet],  axis=1)

# Set the area function as the index
data = temp.set_index(('X','function'), append=True)
data.index.names = ['id','function']

# Drop areas without function (function:"other")
data = data.drop('other', level='function')

data.head()

## Calculate Silhouette scores

In [ ]:
animals = data.columns.get_level_values('mouse').unique()
scores = []
scoresDefaultFunction = []
scoresShuffled = []

for mouse in animals:
    animal = data.xs(mouse, axis=1, level='mouse').copy()

    oldFunctions = animal.index.get_level_values('function')
    hiLowCluster = ['low' if x in ['lateral', 'mPrefrontal'] else 'high' for x in oldFunctions]
    animal['hiLowCluster'] = hiLowCluster

    # Silhouette for the custom clustering
    score = silhouette_score(animal[['diffuseFluo','energy']], animal['hiLowCluster'])
    scores.append(score)

    # Silhouette for functional region clustering
    score = silhouette_score(animal[['diffuseFluo','energy']], animal.index.get_level_values('function'))
    scoresDefaultFunction.append(score)

    # Silhouette for shuffled clustering
    tempScores = []
    rng = np.random.default_rng(123)     # Seed for reproducibility
    for i in range(100):
        # score = silhouette_score(animal[['diffuseFluo','energy']], np.random.permutation(animal['hiLowCluster']))
        score = silhouette_score(animal[['diffuseFluo','energy']], rng.permutation(animal['hiLowCluster']))
        tempScores.append(score)
    scoresShuffled.append(np.mean(tempScores))

# Store everything in a dataFrame
silScores = pd.DataFrame({
    'Low-High WFA':scores,
    'Cortical Subnet.':scoresDefaultFunction,
    'Shuffle':scoresShuffled})

silScores

## Plot the silhouette results

In [ ]:
sns.set_style('white')

# Define colors
cmap = cm.get_cmap("PuBu")
pointColor = cmap(0.85)
barColor = cmap(0.4)

f, ax = plt.subplots(figsize=(2,7))
g = sns.barplot(
    data=silScores,
    color=barColor,
    saturation=0.7,
    errorbar='se')
sns.stripplot(
    data=silScores,
    size=10,
    # color=pointColor,
    color=pointColor,
    edgecolor='white',
    linewidth=0.5,
    zorder=2
)

# Axis customization
plt.axhline(y=0, color='dimgrey', linestyle='-', zorder=1)
g.set_ylabel("Silhouette score", fontsize=20)
g.set_xlabel("Clusters", fontsize=20)
ax.tick_params(labelsize=15)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), rotation=45, ha='right')

sns.despine()


# Statistical analysis
# One way ANOVA
f, pval = f_oneway(silScores['Low-High WFA'], silScores['Cortical Subnet.'], silScores['Shuffle'])
print('='*45)
print(f"One-Way ANOVA - F: {f:.4f}  p-value: {pval:.4}")
print('='*45, end="\n\n")

# Post hocs
variables = ['Low-High WFA','Cortical Subnet.','Shuffle']
pList = []
tList = []
for x in itertools.combinations(variables, r=2):
    print(f"T-test - {x[0]} vs {x[1]}")
    t, p = ttest_ind(silScores[x[0]], silScores[x[1]])
    print(f'\tt: {t:.4f}, p-value: {p:.4}')
    tList.append(t)
    pList.append(p)

# Correct for multiple comparison
_, pAdj, _, _  = multipletests(pList, method='holm-sidak')
_ = [print(f"Adjusted p: {x:.4}") for x in pAdj]

# plt.savefig('Silhouette.svg',bbox_inches="tight")

In [ ]:
pointColor